In [ ]:
from PIL import Image
from IPython.display import display
from sklearn.preprocessing import binarize
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot
import os
import datetime
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras import backend as K
from keras.models import load_model
import tensorflow as tf
import tensorflow_addons as tfa


In [3]:
2**16

65536

In [ ]:

modelname = 'model_epoc_100.h5'
model = load_model(modelname)

path = 'images/'
result = {'precision': 0.0, 'recall': 0, 'f1-score': 0, 'accuracy': 0}
n = 0
files = 0

for image in os.listdir(path):
    print('Processing image ' + image)
    files += 1
    pixels = load_img(path + image, target_size=(256, 512))

    pixels = img_to_array(pixels)

    sat_img, map_img = (pixels[:256, :256], pixels[:256, 256:])
    sat_img = (sat_img - 127.5) / 127.5

    sat_img = K.expand_dims(sat_img, 0)
    map_img = K.expand_dims(map_img, 0)

    y_pred = model.predict(sat_img)
    y_pred = np.array(y_pred, dtype=np.float32)
    y_pred = (y_pred + 1) / 2.0
    y_pred = np.where((y_pred != 1), 0, y_pred)
#'uint8'
#tf.uint8
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    map_img = tf.convert_to_tensor(map_img.numpy()/255, dtype=tf.float32)

    for x in range(256):
        for y in range(256):
            print(x, y, end="\r")
            report = classification_report(map_img.numpy()[0][x][y], y_pred.numpy()[
                0][x][y], zero_division=0, output_dict=True)
            try:
                if report['1.0'] != None:
                    n += 1
                    result['precision'] = result['precision'] + \
                        report['1.0']['precision']
                    result['recall'] = result['recall'] + \
                        report['1.0']['recall']
                    result['f1-score'] = result['f1-score'] + \
                        report['1.0']['f1-score']
                    result['accuracy'] = result['accuracy'] + \
                        report['accuracy']
            except:
                None

result['precision'] = result['precision'] / n
result['recall'] = result['recall'] / n
result['f1-score'] = result['f1-score'] / n
result['accuracy'] = result['accuracy'] / n

print(result, "Pixels:",n)
print(datetime.datetime.now())
